In [181]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
sns.set()

In [182]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [183]:

def preprocess(df):
    cols = []
    for column in df.columns:
        cols.append(column.lower())
    df.columns = cols
    return df


df = preprocess(pd.read_csv('fifa21_train.csv'))
df.head()

,id,name,age,nationality,club,bp,position,team & contract,height,weight,foot,growth,joined,loan date end,value,wage,release clause,contract,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,w/f,sm,a/w,d/w,ir,pac,sho,pas,dri,def,phy,hits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,FC Lausanne-Sport 2015 ~ 2020,"5'9""",161lbs,Right,1,"Jul 1, 2015",NaN,€525K,€4K,€801K,2015 ~ 2020,258,54,47,43,70,44.0,286,61,44.0,55,63,63,346,64,73,61.0,66,82.0,306,62,73.0,71,55,45,290,54,52.0,62.0,68.0,54,54.0,148,49,56,43.0,48,7,12,14,9,6,1682,357,4 ★,2★,High,Medium,1 ★,69,51,63,63,51,60,3,58+1,58+1,58+1,61+0,62+0,62+0,62+0,61+0,63+1,63+1,63+1,63+1,63+1,63+1,63+1,63+1,59+1,59+1,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"Beijing Sinobo Guoan FC Dec 31, 2020 On Loan","6'0""",159lbs,Right,0,"Jan 16, 2015","Dec 31, 2020",€8.5M,€23K,€0,"Dec 31, 2020 On Loan",365,66,79,76,68,76.0,375,83,78.0,72,63,79,404,83,83,88.0,75,75.0,372,74,81.0,75,74,68,313,54,33.0,78.0,72.0,76,70.0,77,35,20,22.0,55,11,7,14,7,16,1961,412,3 ★,4★,High,Low,2 ★,83,75,68,82,33,71,44,77+0,77+0,77+0,77+0,77+0,77+0,77+0,77+0,76+1,76+1,76+1,76+1,68+2,68+2,68+2,76+1,57+2,53+2,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,Al Hilal 2019 ~ 2022,"5'4""",134lbs,Right,0,"Jan 31, 2019",NaN,€9M,€49K,€15.3M,2019 ~ 2022,336,73,76,34,78,75.0,424,85,89.0,91,74,85,424,84,76,93.0,78,93.0,308,79,34.0,75,42,78,332,75,26.0,80.0,78.0,73,82.0,80,23,29,28.0,21,6,3,6,3,3,1925,404,4 ★,4★,High,Medium,2 ★,80,77,78,86,27,56,73,73+2,73+2,73+2,80+0,79+0,79+0,79+0,80+0,80+0,80+0,80+0,79+1,74+2,74+2,74+2,79+1,59+2,56+2,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,Swansea City 2016 ~ 2021,"5'10""",152lbs,Right,13,"Jul 1, 2016",NaN,€275K,€4K,€694K,2016 ~ 2021,242,44,42,58,62,36.0,259,54,41.0,46,57,61,282,54,59,59.0,55,55.0,277,57,60.0,64,58,38,257,61,57.0,31.0,54.0,54,48.0,168,55,58,55.0,42,8,9,6,7,12,1527,329,2 ★,2★,Medium,Medium,1 ★,57,44,54,57,57,60,7,50+2,50+2,50+2,51+0,51+0,51+0,51+0,51+0,53+2,53+2,53+2,53+2,56+2,56+2,56+2,53+2,56+2,58+2,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,US Orléans Loiret Football 2018 ~ 2021,"5'11""",150lbs,Right,8,"Jul 1, 2018",NaN,€725K,€2K,€1.4M,2018 ~ 2021,249,49,37,61,68,34.0,280,64,44.0,45,61,66,324,66,66,61.0,62,69.0,280,61,34.0,81,61,43,294,66,60.0,55.0,64.0,49,58.0,185,58,61,66.0,52,8,9,15,5,15,1664,360,2 ★,3★,Low,Medium,1 ★,66,44,60,64,60,66,4,56+2,56+2,56+2,59+0,59+0,59+0,59+0,59+0,61+2,61+2,61+2,62+2,63+2,63+2,63+2,62+2,64+2,64+2,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2,65


In [184]:
df.isna().sum()

id                      0
name                    0
age                     0
nationality             0
club                   18
bp                      0
position              288
team & contract         0
height                  0
weight                  0
foot                    0
growth                  0
joined                 41
loan date end       11072
value                   0
wage                    0
release clause          0
contract                0
attacking               0
crossing                0
finishing               0
heading accuracy        0
short passing           0
volleys                41
skill                   0
dribbling               0
curve                  41
fk accuracy             0
long passing            0
ball control            0
movement                0
acceleration            0
sprint speed            0
agility                41
reactions               0
balance                41
power                   0
shot power              0
jumping     

In [185]:
df = df.drop(['id','age', 'nationality', 'position', 'club', 'team & contract', 'height', 'weight', 'foot', 'growth', 'joined', 'loan date end', 'value', 'wage', 'release clause', 'contract'], axis=1)

In [186]:
# df = df[df['position'].isna()==False]
df = df[df['composure'].isna()==False]
df.shape

(11422, 85)

In [187]:
# df.isna().sum()

In [188]:
# round(df.isna().sum()/len(df),4)*100  # shows the percentage of null values in a column
# nulls_df = pd.DataFrame(round(df.isna().sum()/len(df),4)*100)
# nulls_df
# nulls_df = nulls_df.reset_index()
# nulls_df
# nulls_df.columns = ['header_name', 'percent_nulls']
# nulls_df

In [189]:
df.columns

Index(['name', 'bp', 'attacking', 'crossing', 'finishing', 'heading accuracy',
       'short passing', 'volleys', 'skill', 'dribbling', 'curve',
       'fk accuracy', 'long passing', 'ball control', 'movement',
       'acceleration', 'sprint speed', 'agility', 'reactions', 'balance',
       'power', 'shot power', 'jumping', 'stamina', 'strength', 'long shots',
       'mentality', 'aggression', 'interceptions', 'positioning', 'vision',
       'penalties', 'composure', 'defending', 'marking', 'standing tackle',
       'sliding tackle', 'goalkeeping', 'gk diving', 'gk handling',
       'gk kicking', 'gk positioning', 'gk reflexes', 'total stats',
       'base stats', 'w/f', 'sm', 'a/w', 'd/w', 'ir', 'pac', 'sho', 'pas',
       'dri', 'def', 'phy', 'hits', 'ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf',
       'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm',
       'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk', 'ova'],
      dtype='object')

In [190]:
df.dtypes

name                 object
bp                   object
attacking             int64
crossing              int64
finishing             int64
heading accuracy      int64
short passing         int64
volleys             float64
skill                 int64
dribbling             int64
curve               float64
fk accuracy           int64
long passing          int64
ball control          int64
movement              int64
acceleration          int64
sprint speed          int64
agility             float64
reactions             int64
balance             float64
power                 int64
shot power            int64
jumping             float64
stamina               int64
strength              int64
long shots            int64
mentality             int64
aggression            int64
interceptions       float64
positioning         float64
vision              float64
penalties             int64
composure           float64
defending             int64
marking               int64
standing tackle     

In [191]:
df['w/f'] = df['w/f'].str.replace('★','').astype(np.int64)
df['sm'] = df['sm'].str.replace('★','').astype(np.int64)
df['ir'] = df['ir'].str.replace('★','').astype(np.int64)



In [192]:
df['hits'] = df['hits'].str.replace('.1K','100')
df['hits'] = df['hits'].str.replace('.2K','200')
df['hits'] = df['hits'].str.replace('.3K','300')
df['hits'] = df['hits'].str.replace('.4K','300')
df['hits'] = df['hits'].str.replace('.5K','500')
df['hits'] = df['hits'].str.replace('.6K','600')
df['hits'] = df['hits'].str.replace('.7K','700')
df['hits'] = df['hits'].str.replace('.8K','800')
df['hits'] = df['hits'].str.replace('.9K','900')
df['hits'] = df['hits'].str.replace('K','000')
df['hits'] = pd.to_numeric(df['hits'])

C:\Users\JT\AppData\Local\Temp\ipykernel_15036\3387961658.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['hits'] = df['hits'].str.replace('.1K','100')
C:\Users\JT\AppData\Local\Temp\ipykernel_15036\3387961658.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['hits'] = df['hits'].str.replace('.2K','200')
C:\Users\JT\AppData\Local\Temp\ipykernel_15036\3387961658.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['hits'] = df['hits'].str.replace('.3K','300')
C:\Users\JT\AppData\Local\Temp\ipykernel_15036\3387961658.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['hits'] = df['hits'].str.replace('.4K','300')
C:\Users\JT\AppData\Local\Temp\ipykernel_15036\3387961658.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['

In [193]:
X_num = df.select_dtypes(include = np.number)
X_cat = df.select_dtypes(include = object)

In [194]:
X_cat.columns

Index(['name', 'bp', 'a/w', 'd/w', 'ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf',
       'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm',
       'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk'],
      dtype='object')

In [195]:
X_cat = X_cat.drop(['name','ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk'],axis=1)
X_cat.head()

,bp,a/w,d/w
0,CM,High,Medium
1,ST,High,Low
2,CAM,High,Medium
3,CDM,Medium,Medium
4,CDM,Low,Medium


### Normalizing numerical data

In [196]:
from sklearn.preprocessing import MinMaxScaler
MinMaxtransformer = MinMaxScaler().fit(X_num)
X_num_normalized = MinMaxtransformer.transform(X_num)
print(X_num_normalized.shape)
X_num_normalized = pd.DataFrame(X_num_normalized,columns=X_num.columns)
X_num_normalized.head()

(11422, 54)


,attacking,crossing,finishing,heading accuracy,short passing,volleys,skill,dribbling,curve,fk accuracy,long passing,ball control,movement,acceleration,sprint speed,agility,reactions,balance,power,shot power,jumping,stamina,strength,long shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing tackle,sliding tackle,goalkeeping,gk diving,gk handling,gk kicking,gk positioning,gk reflexes,total stats,base stats,w/f,sm,ir,pac,sho,pas,dri,def,phy,hits,ova
0,0.546835,0.545455,0.478261,0.431818,0.710843,0.465116,0.569087,0.615385,0.444444,0.561798,0.642857,0.637363,0.663818,0.607143,0.729412,0.573171,0.591549,0.8125,0.556270,0.602410,0.685714,0.694118,0.454545,0.455556,0.654596,0.517241,0.552941,0.645161,0.682353,0.547619,0.500000,0.518219,0.516854,0.595238,0.440476,0.084309,0.056818,0.116279,0.139535,0.078652,0.045455,0.604577,0.492366,0.75,0.25,0.00,0.614286,0.447368,0.558824,0.522388,0.506494,0.515625,0.000000,0.395833
1,0.817722,0.681818,0.826087,0.806818,0.686747,0.837209,0.777518,0.857143,0.822222,0.752809,0.642857,0.813187,0.829060,0.833333,0.847059,0.902439,0.718310,0.7250,0.768489,0.746988,0.800000,0.741176,0.701299,0.711111,0.718663,0.517241,0.329412,0.817204,0.729412,0.809524,0.690476,0.230769,0.359551,0.166667,0.190476,0.100703,0.102273,0.058140,0.139535,0.056180,0.159091,0.781945,0.702290,0.50,0.75,0.25,0.814286,0.763158,0.632353,0.805970,0.272727,0.687500,0.004458,0.666667
2,0.744304,0.761364,0.793478,0.329545,0.807229,0.825581,0.892272,0.879121,0.944444,0.966292,0.773810,0.879121,0.886040,0.845238,0.764706,0.963415,0.760563,0.9500,0.562701,0.807229,0.128571,0.741176,0.285714,0.822222,0.771588,0.758621,0.247059,0.838710,0.800000,0.773810,0.833333,0.242915,0.224719,0.273810,0.261905,0.021077,0.045455,0.011628,0.046512,0.011236,0.011364,0.759059,0.671756,0.75,0.75,0.25,0.771429,0.789474,0.779412,0.865672,0.194805,0.453125,0.007611,0.729167
3,0.506329,0.431818,0.423913,0.602273,0.614458,0.372093,0.505855,0.538462,0.411111,0.460674,0.571429,0.615385,0.481481,0.488095,0.564706,0.548780,0.436620,0.4750,0.463023,0.542169,0.500000,0.611765,0.493506,0.377778,0.562674,0.597701,0.611765,0.311828,0.517647,0.547619,0.428571,0.599190,0.584270,0.619048,0.583333,0.070258,0.068182,0.081395,0.046512,0.056180,0.113636,0.506039,0.385496,0.25,0.25,0.00,0.442857,0.355263,0.426471,0.432836,0.584416,0.515625,0.000435,0.291667
4,0.524051,0.488636,0.369565,0.636364,0.686747,0.348837,0.555035,0.648352,0.444444,0.449438,0.619048,0.670330,0.601140,0.630952,0.647059,0.573171,0.535211,0.6500,0.472669,0.590361,0.128571,0.811765,0.532468,0.433333,0.665738,0.655172,0.647059,0.569892,0.635294,0.488095,0.547619,0.668016,0.617978,0.654762,0.714286,0.093677,0.068182,0.081395,0.151163,0.033708,0.147727,0.593134,0.503817,0.25,0.50,0.00,0.571429,0.355263,0.514706,0.537313,0.623377,0.609375,0.000109,0.416667


In [197]:
X_cat.head()

,bp,a/w,d/w
0,CM,High,Medium
1,ST,High,Low
2,CAM,High,Medium
3,CDM,Medium,Medium
4,CDM,Low,Medium


### Encoding categorical variables

In [198]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first').fit(X_cat)
encoded = encoder.transform(X_cat).toarray()
encoded
cols = encoder.get_feature_names_out(input_features=X_cat.columns)
cols
onehot_encoded = pd.DataFrame(encoded, columns=cols)
onehot_encoded.head()

,bp_CB,bp_CDM,bp_CF,bp_CM,bp_GK,bp_LB,bp_LM,bp_LW,bp_LWB,bp_RB,bp_RM,bp_RW,bp_RWB,bp_ST,a/w_Low,a/w_Medium,d/w_Low,d/w_Medium
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [199]:
onehot_encoded.shape

(11422, 18)

In [200]:
# corr = X_num_normalized.corr().loc[:,['ova']]
# fig, ax = plt.subplots(figsize = (5,15))

# ax = sns.heatmap(corr.sort_values(by=['ova'], ascending = False), ax = ax, annot = True, cbar = True, cmap='Reds')
# ax.set_xticklabels(ax.xaxis.get_ticklabels(), fontsize = 10)
# ax.set_yticklabels(ax.yaxis.get_ticklabels(), fontsize = 10)

### X-y split

In [201]:
X_normalized = pd.concat([X_num_normalized, onehot_encoded], axis=1)
X_normalized.shape

(11422, 72)

In [202]:
y = df['ova']

# X = X.drop(['total claim amount'], axis=1)
X_normalized = X_normalized.drop(['ova'], axis=1)

### Testing the model

In [203]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)


In [204]:
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
lm = linear_model.LinearRegression()

In [205]:
lm.fit(X_train,y_train)

LinearRegression()

In [206]:
from sklearn.metrics import r2_score
predictions = lm.predict(X_train)
r2_score(y_train, predictions)

0.9151009056775905

In [207]:
predictions_test = lm.predict(X_test)
r2_score(y_test, predictions_test)

0.9057458064573707

In [208]:
y_test[:5]

4971     65
10174    66
5002     60
5273     63
2646     67
Name: ova, dtype: int64

In [209]:
predictions_test[:5]

array([63.73698733, 67.34378694, 59.74934651, 64.02371491, 65.78793755])

In [210]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
mse=mean_squared_error(y_test,predictions_test)
mse

4.415163039962316

In [211]:
rmse = np.sqrt(mean_squared_error(y_test,predictions_test))
rmse

2.1012289356379794

In [212]:
mae = mean_absolute_error(y_test, predictions_test)
mae

1.6136290568262366

In [213]:
id_max = np.argmax(lm.predict(X_test)) # find id of the maximum predicted label
print(X_normalized.loc[id_max])

attacking           0.701266
crossing            0.840909
finishing           0.565217
heading accuracy    0.556818
short passing       0.746988
volleys             0.616279
skill               0.723653
dribbling           0.758242
curve               0.777778
fk accuracy         0.617978
long passing        0.726190
ball control        0.758242
movement            0.834758
acceleration        0.892857
sprint speed        0.752941
agility             0.865854
reactions           0.690141
balance             0.850000
power               0.668810
shot power          0.650602
jumping             0.757143
stamina             0.776471
strength            0.584416
long shots          0.555556
mentality           0.821727
aggression          0.747126
interceptions       0.788235
positioning         0.741935
vision              0.752941
penalties           0.607143
composure           0.678571
defending           0.785425
marking             0.752809
standing tackle     0.785714
sliding tackle